In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings

In [2]:
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")
sample_sub = pd.read_csv("input/sample_submission.csv")

In [3]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
use_features = [
    "EXT_SOURCE_1",
    "EXT_SOURCE_2", 
    "EXT_SOURCE_3",
    
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "AMT_INCOME_TOTAL",
    "AMT_GOODS_PRICE",
    
    "OWN_CAR_AGE",

    "DAYS_BIRTH",
    "DAYS_EMPLOYED",
    "CODE_GENDER",
    "CNT_CHILDREN",
    
    "NAME_EDUCATION_TYPE",
    "NAME_INCOME_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_CONTRACT_TYPE",
    
    "ORGANIZATION_TYPE",
    "REGION_POPULATION_RELATIVE",
]

target = train["TARGET"].values

train = train[use_features]
train["TARGET"] = target
test = test[use_features]

In [5]:
train

,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,AMT_CREDIT,AMT_ANNUITY,AMT_INCOME_TOTAL,AMT_GOODS_PRICE,OWN_CAR_AGE,DAYS_BIRTH,DAYS_EMPLOYED,CODE_GENDER,CNT_CHILDREN,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_CONTRACT_TYPE,ORGANIZATION_TYPE,REGION_POPULATION_RELATIVE,TARGET
0,NaN,0.372591,NaN,755190.0,36328.5,112500.0,675000.0,NaN,-9233,-878,F,0,Higher education,Working,Married,Cash loans,School,0.010032,0
1,NaN,0.449567,0.553165,585000.0,16893.0,225000.0,585000.0,NaN,-20148,365243,F,0,Secondary / secondary special,Pensioner,Married,Cash loans,XNA,0.008019,0
2,NaN,0.569503,NaN,334152.0,18256.5,54000.0,270000.0,NaN,-18496,-523,F,0,Secondary / secondary special,State servant,Married,Cash loans,Postal,0.004960,0
3,NaN,0.105235,0.767523,152820.0,8901.0,67500.0,135000.0,NaN,-24177,365243,F,0,Lower secondary,Pensioner,Widow,Cash loans,XNA,0.005002,0
4,0.342344,0.202490,0.669057,271066.5,21546.0,157500.0,234000.0,NaN,-10685,-697,M,0,Secondary / secondary special,Commercial associate,Married,Cash loans,Business Entity Type 3,0.006296,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171197,NaN,0.404560,0.768808,404325.0,20772.0,83250.0,337500.0,21.0,-20529,-3059,M,0,Secondary / secondary special,Working,Married,Cash loans,Agriculture,0.031329,0
171198,NaN,0.608542,NaN,601470.0,29065.5,247500.0,450000.0,1.0,-22083,-129,F,0,Secondary / secondary special,Working,Single / not married,Cash loans,Industry: type 3,0.010006,0
171199,NaN,0.664305,0.758393,1237684.5,49216.5,292500.0,1138500.0,NaN,-11053,-2536,F,2,Higher education,State servant,Married,Cash loans,School,0.006629,0
171200,0.210918,0.627050,NaN,239850.0,25186.5,112500.0,225000.0,7.0,-8505,-165,M,0,Secondary / secondary special,Commercial associate,Single / not married,Cash loans,Business Entity Type 3,0.009657,0


In [6]:
# Numerization of ORGANIZATION_TYPE (Count Encoding）
organization_ce = train["ORGANIZATION_TYPE"].value_counts()
train["ORGANIZATION_TYPE"] = train["ORGANIZATION_TYPE"].map(organization_ce)
test["ORGANIZATION_TYPE"] = test["ORGANIZATION_TYPE"].map(organization_ce)

In [7]:
# Numerization of ORGANIZATION_TYPE (Count Encoding）
nameFamStatus = train["NAME_FAMILY_STATUS"].value_counts()
train["NAME_FAMILY_STATUS"] = train["NAME_FAMILY_STATUS"].map(nameFamStatus)
test["NAME_FAMILY_STATUS"] = test["NAME_FAMILY_STATUS"].map(nameFamStatus)

In [8]:
# Numerization of ORGANIZATION_TYPE (Count Encoding）
incomeType = train["NAME_INCOME_TYPE"].value_counts()
train["NAME_INCOME_TYPE"] = train["NAME_INCOME_TYPE"].map(incomeType)
test["NAME_INCOME_TYPE"] = test["NAME_INCOME_TYPE"].map(incomeType)

In [9]:
# Numerization of ORGANIZATION_TYPE (Count Encoding）
eduType = train["NAME_EDUCATION_TYPE"].value_counts()
train["NAME_EDUCATION_TYPE"] = train["NAME_EDUCATION_TYPE"].map(eduType)
test["NAME_EDUCATION_TYPE"] = test["NAME_EDUCATION_TYPE"].map(eduType)

In [10]:
# Treat values above 60 (outliers) in OWN_CAR_AGE as missing values
train.loc[train["REGION_POPULATION_RELATIVE"] >= 0.07, "OWN_CAR_AGE"] = np.nan
test.loc[test["REGION_POPULATION_RELATIVE"] >= 0.07, "OWN_CAR_AGE"] = np.nan

In [11]:
# Numerization of NAME_CONTRACT_TYPE（Label Encoding）
train["NAME_CONTRACT_TYPE"].replace({'Cash loans': 0, 'Revolving loans': 1}, inplace=True)
test["NAME_CONTRACT_TYPE"].replace({'Cash loans': 0, 'Revolving loans': 1}, inplace=True)

In [12]:
# Numerization of NAME_CONTRACT_TYPE（Label Encoding）
train["CODE_GENDER"].replace({'XNA': 0, 'F': 1, 'M' : 2 }, inplace=True)
test["CODE_GENDER"].replace({'XNA': 0, 'F': 1, 'M' : 2}, inplace=True)

In [13]:
train["EXT_SOURCE_2"].fillna(train["EXT_SOURCE_2"].mean(), inplace=True)
test["EXT_SOURCE_2"].fillna(train["EXT_SOURCE_2"].mean(), inplace=True) 

In [14]:
#train["EXT_SOURCE_1"].fillna(train["EXT_SOURCE_1"].mean(), inplace=True)
#test["EXT_SOURCE_1"].fillna(train["EXT_SOURCE_1"].mean(), inplace=True) 

In [15]:
#train["EXT_SOURCE_3"].fillna(train["EXT_SOURCE_3"].mean(), inplace=True)
#test["EXT_SOURCE_3"].fillna(train["EXT_SOURCE_3"].mean(), inplace=True) 

try to feture engginering later to have better data

In [16]:
# Treat values above 60 (outliers) in OWN_CAR_AGE as missing values
train.loc[train["OWN_CAR_AGE"] >= 60, "OWN_CAR_AGE"] = np.nan
test.loc[test["OWN_CAR_AGE"] >= 60, "OWN_CAR_AGE"] = np.nan

In [17]:
# Divide OWN_CAR_AGE into groups
train["OWN_CAR_AGE"] = train["OWN_CAR_AGE"] // 10
test["OWN_CAR_AGE"] = test["OWN_CAR_AGE"] // 10

In [18]:
# Apply One Hot Encoding to OWN_CAR_AGE
train_car_age_ohe = pd.get_dummies(train["OWN_CAR_AGE"]).add_prefix("OWN_CAR_AGE_")
test_car_age_ohe = pd.get_dummies(test["OWN_CAR_AGE"]).add_prefix("OWN_CAR_AGE_")

# Add the one hot encoded columns to train/test
train = pd.concat([train, train_car_age_ohe], axis=1)
test = pd.concat([test, test_car_age_ohe], axis=1)

# Remove original OWN_CAR_AGE
train.drop('OWN_CAR_AGE', axis=1, inplace=True)
test.drop('OWN_CAR_AGE', axis=1, inplace=True)

In [19]:
test

,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,AMT_CREDIT,AMT_ANNUITY,AMT_INCOME_TOTAL,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,CODE_GENDER,...,NAME_FAMILY_STATUS,NAME_CONTRACT_TYPE,ORGANIZATION_TYPE,REGION_POPULATION_RELATIVE,OWN_CAR_AGE_0.0,OWN_CAR_AGE_1.0,OWN_CAR_AGE_2.0,OWN_CAR_AGE_3.0,OWN_CAR_AGE_4.0,OWN_CAR_AGE_5.0
0,NaN,0.720416,NaN,961146.0,28233.0,144000.0,688500.0,-12108,-2372,1,...,109323,0,3866,0.025164,False,False,False,False,False,False
1,NaN,0.287306,NaN,296280.0,16069.5,103500.0,225000.0,-17907,-1712,1,...,109323,0,4991,0.007020,False,False,False,False,False,False
2,NaN,0.352456,0.389339,183694.5,11236.5,180000.0,139500.0,-15221,-553,1,...,25285,0,4336,0.006852,False,False,False,False,False,False
3,NaN,0.470384,0.217629,450000.0,22500.0,225000.0,450000.0,-11217,-1438,1,...,109323,1,37943,0.035792,False,False,False,False,False,False
4,0.269931,0.373133,NaN,545040.0,26640.0,144000.0,450000.0,-11415,-2362,1,...,109323,0,37943,0.020713,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61495,0.263678,0.018172,0.307737,1288350.0,37800.0,315000.0,1125000.0,-11430,-792,1,...,109323,0,4336,0.007020,False,True,False,False,False,False
61496,NaN,0.668578,0.434733,273636.0,15408.0,90000.0,247500.0,-17181,-839,1,...,109323,0,21340,0.006671,False,False,False,False,False,False
61497,0.510226,0.574151,NaN,291384.0,26725.5,144000.0,270000.0,-14515,-722,1,...,25285,0,37943,0.018801,False,False,False,False,False,False
61498,0.353295,0.226714,NaN,746280.0,59094.0,193500.0,675000.0,-16914,-8756,1,...,109323,0,9272,0.002042,False,False,False,False,False,False


In [20]:
train

,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,AMT_CREDIT,AMT_ANNUITY,AMT_INCOME_TOTAL,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,CODE_GENDER,...,NAME_CONTRACT_TYPE,ORGANIZATION_TYPE,REGION_POPULATION_RELATIVE,TARGET,OWN_CAR_AGE_0.0,OWN_CAR_AGE_1.0,OWN_CAR_AGE_2.0,OWN_CAR_AGE_3.0,OWN_CAR_AGE_4.0,OWN_CAR_AGE_5.0
0,NaN,0.372591,NaN,755190.0,36328.5,112500.0,675000.0,-9233,-878,1,...,0,4991,0.010032,0,False,False,False,False,False,False
1,NaN,0.449567,0.553165,585000.0,16893.0,225000.0,585000.0,-20148,365243,1,...,0,30898,0.008019,0,False,False,False,False,False,False
2,NaN,0.569503,NaN,334152.0,18256.5,54000.0,270000.0,-18496,-523,1,...,0,1185,0.004960,0,False,False,False,False,False,False
3,NaN,0.105235,0.767523,152820.0,8901.0,67500.0,135000.0,-24177,365243,1,...,0,30898,0.005002,0,False,False,False,False,False,False
4,0.342344,0.202490,0.669057,271066.5,21546.0,157500.0,234000.0,-10685,-697,2,...,0,37943,0.006296,1,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171197,NaN,0.404560,0.768808,404325.0,20772.0,83250.0,337500.0,-20529,-3059,2,...,0,1338,0.031329,0,False,False,True,False,False,False
171198,NaN,0.608542,NaN,601470.0,29065.5,247500.0,450000.0,-22083,-129,1,...,0,1833,0.010006,0,True,False,False,False,False,False
171199,NaN,0.664305,0.758393,1237684.5,49216.5,292500.0,1138500.0,-11053,-2536,1,...,0,4991,0.006629,0,False,False,False,False,False,False
171200,0.210918,0.627050,NaN,239850.0,25186.5,112500.0,225000.0,-8505,-165,2,...,0,37943,0.009657,0,True,False,False,False,False,False
